# Part 1.2:  Redfin-SimpleMap-Zillow Housing Price Clean & Merge 
This Notebook is divided into 3 sections
* **Redfin Dataset:** Redfin dataset is loaded, cleaned, and transformed
* **SimpleMaps-Redfin:** SimpleMap dataset which contains CountyName feature needed for the Redfin dataset is loaded, cleaned and merged with Redfin dataset
* **Merge Zillow-Redfin:** Final merge(appending rows) of the redfin-simplemap and clean zillow dataset 


### Extract**
Data Source Data & Documentation:
* Redfin : https://www.redfin.com/news/data-center/ houseprices were extracted on zipcodes: csv file
* SimpleMaps: https://simplemaps.com/data/us-counties 
The redfin data does not have cities and county names so we will match the zipcodes in the Redfin dataset to the city and county names in the SimpleMaps dataset. 
* Clean Zillow Data : cleaned csv file from notebook zillow_housing_final_cleaned.ipynb
The data is extracted from csvs in each section. 

### Transform

##### Redfin Dataset
* Uravel column "period_end" and split into mulitple columns
* Drop unneeded columns
* Filter by only house prices for 2021
* Create a dataframe “price_df” that has per postal_code the mean house price
* Drop columns from the original redfin dataframe “df" and drop duplicates
* Merge dataframes "price_df" and "df"

##### SimpleMaps Dataset and Redfin Data Merge`
* Fill in zeros to the begining of zipcodes to make them 5 digits
* Drop unnecessary columns
* Rename zip as postal_code
* Merge Redfin and SimpleMaps dataframes
* Drop unnecessary columns
* Rename columns - state_id: "State", city:"City", county_name:"CountyName
* Reorder columns of dataframe and save as redfin_df

##### Merge 
* Append Zillow dataframe "zillow_df" and Redfin-SimpleMaps merged dataframe "redfin_df" as merge_df

### Load
Read merged, cleaned and tranformed merge_df dataframe as csv

The final data that will be used to merge with Yelp API data has the following structure:
* postal_code : zipcodes in United States
* State	: states in United States
* City  : cities in United States
* CountyName : counties in United States	
* 2021 : Median house price per zipcode for Single Family homes in 2021

In [67]:
#import dependencies
import pandas as pd
from functools import reduce

# Redfin Housing Dataset

## Extract

In [2]:
#load data
redfin = pd.read_csv('redfin_raw.tsv000', sep='\t')


In [11]:
df = redfin.copy()

In [12]:
#list of columns in dataframe
df.columns.to_list()

['period_begin',
 'period_end',
 'period_duration',
 'region_type',
 'region_type_id',
 'table_id',
 'is_seasonally_adjusted',
 'region',
 'city',
 'state',
 'state_code',
 'property_type',
 'property_type_id',
 'median_sale_price',
 'median_sale_price_mom',
 'median_sale_price_yoy',
 'median_list_price',
 'median_list_price_mom',
 'median_list_price_yoy',
 'median_ppsf',
 'median_ppsf_mom',
 'median_ppsf_yoy',
 'median_list_ppsf',
 'median_list_ppsf_mom',
 'median_list_ppsf_yoy',
 'homes_sold',
 'homes_sold_mom',
 'homes_sold_yoy',
 'pending_sales',
 'pending_sales_mom',
 'pending_sales_yoy',
 'new_listings',
 'new_listings_mom',
 'new_listings_yoy',
 'inventory',
 'inventory_mom',
 'inventory_yoy',
 'months_of_supply',
 'months_of_supply_mom',
 'months_of_supply_yoy',
 'median_dom',
 'median_dom_mom',
 'median_dom_yoy',
 'avg_sale_to_list',
 'avg_sale_to_list_mom',
 'avg_sale_to_list_yoy',
 'sold_above_list',
 'sold_above_list_mom',
 'sold_above_list_yoy',
 'price_drops',
 'price_dro

In [13]:
df.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2020-05-01,2020-07-31,90,zip code,2,2649,f,Zip Code: 07418,NaN,New Jersey,...,0.2,NaN,NaN,NaN,0.4,0.0,NaN,"Newark, NJ",35084,2022-03-13 14:34:13
1,2014-12-01,2015-02-28,90,zip code,2,41969,f,Zip Code: 85123,NaN,Arizona,...,0.0,NaN,NaN,NaN,0.0,-1.0,NaN,"Phoenix, AZ",38060,2022-03-13 14:34:13
2,2021-12-01,2022-02-28,90,zip code,2,31979,f,Zip Code: 74731,NaN,Oklahoma,...,0.5,NaN,NaN,NaN,1.0,NaN,1.0,"Durant, OK",20460,2022-03-13 14:34:13
3,2021-03-01,2021-05-31,90,zip code,2,19296,f,Zip Code: 45784,NaN,Ohio,...,0.6,NaN,NaN,NaN,0.5,0.0,0.5,"Marietta, OH",31930,2022-03-13 14:34:13
4,2019-02-01,2019-04-30,90,zip code,2,1981,f,Zip Code: 05738,NaN,Vermont,...,-1.0,NaN,NaN,NaN,0.0,NaN,NaN,"Rutland, VT",40860,2022-03-13 14:34:13


In [14]:
df.shape

(5446179, 58)

In [15]:
#datatypes
df.dtypes

period_begin                       object
period_end                         object
period_duration                     int64
region_type                        object
region_type_id                      int64
table_id                            int64
is_seasonally_adjusted             object
region                             object
city                              float64
state                              object
state_code                         object
property_type                      object
property_type_id                    int64
median_sale_price                 float64
median_sale_price_mom             float64
median_sale_price_yoy             float64
median_list_price                 float64
median_list_price_mom             float64
median_list_price_yoy             float64
median_ppsf                       float64
median_ppsf_mom                   float64
median_ppsf_yoy                   float64
median_list_ppsf                  float64
median_list_ppsf_mom              

## Transform

####  Uravel column "period_end" and split into mulitple columns

In [16]:
# uravel period_end and split into mulitple columns
df[["year", "month","date"]]=df["period_end"].str.split('-', expand=True)

In [17]:
df.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated,year,month,date
0,2020-05-01,2020-07-31,90,zip code,2,2649,f,Zip Code: 07418,NaN,New Jersey,...,NaN,0.4,0.0,NaN,"Newark, NJ",35084,2022-03-13 14:34:13,2020,07,31
1,2014-12-01,2015-02-28,90,zip code,2,41969,f,Zip Code: 85123,NaN,Arizona,...,NaN,0.0,-1.0,NaN,"Phoenix, AZ",38060,2022-03-13 14:34:13,2015,02,28
2,2021-12-01,2022-02-28,90,zip code,2,31979,f,Zip Code: 74731,NaN,Oklahoma,...,NaN,1.0,NaN,1.0,"Durant, OK",20460,2022-03-13 14:34:13,2022,02,28
3,2021-03-01,2021-05-31,90,zip code,2,19296,f,Zip Code: 45784,NaN,Ohio,...,NaN,0.5,0.0,0.5,"Marietta, OH",31930,2022-03-13 14:34:13,2021,05,31
4,2019-02-01,2019-04-30,90,zip code,2,1981,f,Zip Code: 05738,NaN,Vermont,...,NaN,0.0,NaN,NaN,"Rutland, VT",40860,2022-03-13 14:34:13,2019,04,30


#### Drop unneeded columns

In [18]:
#drop unneeded columns
df = df.drop(columns=['period_begin',	'period_end',	'period_duration',	'region_type',	'region_type_id',
                      'table_id',	'is_seasonally_adjusted','city','state_code','property_type_id','median_sale_price_mom',
                      'median_sale_price_yoy',	'median_list_price',	'median_list_price_mom',	'median_list_price_yoy',
                      'median_ppsf',	'median_ppsf_mom',	'median_ppsf_yoy',	'median_list_ppsf',
                      'median_list_ppsf_mom','median_list_ppsf_yoy','homes_sold_mom',	'homes_sold_yoy',
                      'pending_sales',	'pending_sales_mom','pending_sales_yoy',	'new_listings',	'new_listings_mom',
                      'new_listings_yoy',	'inventory',	'inventory_mom',	'inventory_yoy',	'months_of_supply',
                      'months_of_supply_mom',	'months_of_supply_yoy',	'median_dom',	'median_dom_mom',	
                      'median_dom_yoy',	'avg_sale_to_list',	'avg_sale_to_list_mom',	'avg_sale_to_list_yoy',	
                      'sold_above_list',	'sold_above_list_mom',	'sold_above_list_yoy',	'price_drops',	
                      'price_drops_mom',	'price_drops_yoy',	'off_market_in_two_weeks',	
                      'off_market_in_two_weeks_mom',	'off_market_in_two_weeks_yoy',	'parent_metro_region',	
                      'parent_metro_region_metro_code',	'last_updated', 'homes_sold', 'property_type'], axis=1)

In [20]:
df.head()

,region,state,median_sale_price,year,month,date
0,Zip Code: 07418,New Jersey,266000.0,2020,07,31
1,Zip Code: 85123,Arizona,106000.0,2015,02,28
2,Zip Code: 74731,Oklahoma,322500.0,2022,02,28
3,Zip Code: 45784,Ohio,205000.0,2021,05,31
4,Zip Code: 05738,Vermont,346500.0,2019,04,30


In [26]:
#unravel the region column into type and postal code
df[["type", "postal_code"]]=df["region"].str.split(":", expand=True)
df.head()

,region,state,median_sale_price,year,month,date,type,postal_code
3,Zip Code: 45784,Ohio,205000.0,2021,05,31,Zip Code,45784
9,Zip Code: 76114,Texas,219500.0,2021,03,31,Zip Code,76114
16,Zip Code: 91405,California,711250.0,2021,06,30,Zip Code,91405
17,Zip Code: 60157,Illinois,385000.0,2021,02,28,Zip Code,60157
22,Zip Code: 50309,Iowa,241900.0,2021,03,31,Zip Code,50309


#### Filter by only house prices for 2021

In [27]:
#filter by only house prices for 2021
df = df[df["year"]=="2021"]

In [28]:
df.head()

,region,state,median_sale_price,year,month,date,type,postal_code
3,Zip Code: 45784,Ohio,205000.0,2021,05,31,Zip Code,45784
9,Zip Code: 76114,Texas,219500.0,2021,03,31,Zip Code,76114
16,Zip Code: 91405,California,711250.0,2021,06,30,Zip Code,91405
17,Zip Code: 60157,Illinois,385000.0,2021,02,28,Zip Code,60157
22,Zip Code: 50309,Iowa,241900.0,2021,03,31,Zip Code,50309


In [29]:
df.shape

(604839, 8)

#### Create a dataframe that has per postal_code the mean house price

In [30]:
#create a dataframe that has per postal_code the mean house price
price = df.groupby(["postal_code"]).mean()["median_sale_price"].round(0)

price_df = pd.DataFrame({"2021": price})
price_df.head()

,2021
postal_code,
01001,247926.0
01002,394252.0
01005,326934.0
01007,332740.0
01008,258562.0


In [31]:
price_df.reset_index()

,postal_code,2021
0,01001,247926.0
1,01002,394252.0
2,01005,326934.0
3,01007,332740.0
4,01008,258562.0
...,...,...
17765,99705,276592.0
17766,99709,246938.0
17767,99712,298337.0
17768,99714,270312.0


#### Drop columns from the original redfin dataframe df and drop duplicates

In [32]:
df.columns

Index(['region', 'state', 'median_sale_price', 'year', 'month', 'date', 'type',
       'postal_code'],
      dtype='object')

In [33]:
df = df.drop(columns=['median_sale_price', 'year',
       'month', 'date', 'type', 'region'])

In [34]:
#drop duplicate postal_code
df = df.drop_duplicates(subset=["postal_code"])

In [35]:
df.shape

(17770, 2)

In [36]:
df.head()

,state,postal_code
3,Ohio,45784
9,Texas,76114
16,California,91405
17,Illinois,60157
22,Iowa,50309


#### Merge dataframes "price_df" and "df"

In [37]:
# Merge dataframes
frames = [df, price_df]
redfin_df = reduce(lambda left,right: pd.merge(left,right,on='postal_code'), frames)
print(redfin_df.shape)
redfin_df.head()

(17770, 3)


,state,postal_code,2021
0,Ohio,45784,248810.0
1,Texas,76114,261764.0
2,California,91405,663543.0
3,Illinois,60157,419458.0
4,Iowa,50309,230289.0


In [38]:
redfin_df.state.value_counts()

California        1464
New York          1285
Texas             1149
Pennsylvania       985
Ohio               959
Florida            959
Illinois           662
Michigan           621
New Jersey         589
Massachusetts      499
Wisconsin          490
Minnesota          471
Georgia            455
North Carolina     444
Washington         435
Tennessee          398
Maryland           397
Virginia           381
Missouri           361
Oregon             326
Oklahoma           322
Arizona            317
Connecticut        286
Colorado           284
Arkansas           273
South Carolina     267
Alabama            221
New Hampshire      219
Indiana            217
Louisiana          216
Utah               184
Kentucky           181
Maine              164
Vermont            160
Nevada             149
Iowa               148
Mississippi        142
Nebraska           140
Idaho              122
Hawaii              82
Rhode Island        78
Kansas              68
New Mexico          48
West Virgin

In [39]:
redfin_df["postal_code"].nunique()

17770

# SimpleMaps Dataset & Redfin Data  Merge

Getting corresponding city and county names from simplemaps dataset

## Load

In [41]:
#load simplemap dataset
simplemap_df = pd.read_csv("uszips1.csv")
print(simplemap_df.shape)
simplemap_df.head()

(33120, 18)


,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,601,18.18005,-66.75218,Adjuntas,PR,Puerto Rico,True,NaN,17113.0,102.7,72001,Adjuntas,"{""72001"": ""99.43"", ""72141"": ""0.57""}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,602,18.36074,-67.17519,Aguada,PR,Puerto Rico,True,NaN,37751.0,476.0,72003,Aguada,"{""72003"": ""100""}",Aguada,72003,False,False,America/Puerto_Rico
2,603,18.45440,-67.12201,Aguadilla,PR,Puerto Rico,True,NaN,47081.0,574.9,72005,Aguadilla,"{""72005"": ""100""}",Aguadilla,72005,False,False,America/Puerto_Rico
3,606,18.16721,-66.93828,Maricao,PR,Puerto Rico,True,NaN,6392.0,58.3,72093,Maricao,"{""72093"": ""94.88"", ""72153"": ""3.78"", ""72121"": ""...",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,610,18.29032,-67.12244,Anasco,PR,Puerto Rico,True,NaN,26686.0,286.9,72011,Añasco,"{""72011"": ""99.45"", ""72003"": ""0.55""}",Añasco|Aguada,72011|72003,False,False,America/Puerto_Rico


## Transform

#### Fill in zeros to the begining of zipcodes to make them 5 digits


In [42]:
simplemap_df['zip'] = simplemap_df['zip'].map(lambda x: str(x).zfill(5))

In [43]:
simplemap_df.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,00601,18.18005,-66.75218,Adjuntas,PR,Puerto Rico,True,NaN,17113.0,102.7,72001,Adjuntas,"{""72001"": ""99.43"", ""72141"": ""0.57""}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,00602,18.36074,-67.17519,Aguada,PR,Puerto Rico,True,NaN,37751.0,476.0,72003,Aguada,"{""72003"": ""100""}",Aguada,72003,False,False,America/Puerto_Rico
2,00603,18.45440,-67.12201,Aguadilla,PR,Puerto Rico,True,NaN,47081.0,574.9,72005,Aguadilla,"{""72005"": ""100""}",Aguadilla,72005,False,False,America/Puerto_Rico
3,00606,18.16721,-66.93828,Maricao,PR,Puerto Rico,True,NaN,6392.0,58.3,72093,Maricao,"{""72093"": ""94.88"", ""72153"": ""3.78"", ""72121"": ""...",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,00610,18.29032,-67.12244,Anasco,PR,Puerto Rico,True,NaN,26686.0,286.9,72011,Añasco,"{""72011"": ""99.45"", ""72003"": ""0.55""}",Añasco|Aguada,72011|72003,False,False,America/Puerto_Rico


#### Drop unnecessary columns

In [44]:
#drop unnecessary columns
simplemap_df.columns

Index(['zip', 'lat', 'lng', 'city', 'state_id', 'state_name', 'zcta',
       'parent_zcta', 'population', 'density', 'county_fips', 'county_name',
       'county_weights', 'county_names_all', 'county_fips_all', 'imprecise',
       'military', 'timezone'],
      dtype='object')

In [45]:
simplemap_df = simplemap_df.drop(columns=['lat', 'lng', 'zcta',
       'parent_zcta', 'population', 'density', 'county_fips',
       'county_weights', 'county_names_all', 'county_fips_all', 'imprecise',
       'military', 'timezone'])

In [46]:
simplemap_df.head()

,zip,city,state_id,state_name,county_name
0,00601,Adjuntas,PR,Puerto Rico,Adjuntas
1,00602,Aguada,PR,Puerto Rico,Aguada
2,00603,Aguadilla,PR,Puerto Rico,Aguadilla
3,00606,Maricao,PR,Puerto Rico,Maricao
4,00610,Anasco,PR,Puerto Rico,Añasco


#### Rename zip as postal_code

In [47]:
# Rename zip as postal_code
simplemap_df = simplemap_df.rename(columns={"zip": "postal_code"})
simplemap_df.head()

,postal_code,city,state_id,state_name,county_name
0,00601,Adjuntas,PR,Puerto Rico,Adjuntas
1,00602,Aguada,PR,Puerto Rico,Aguada
2,00603,Aguadilla,PR,Puerto Rico,Aguadilla
3,00606,Maricao,PR,Puerto Rico,Maricao
4,00610,Anasco,PR,Puerto Rico,Añasco


In [48]:
simplemap_df.dtypes

postal_code    object
city           object
state_id       object
state_name     object
county_name    object
dtype: object

#### Merge Redfin and SimpleMaps dataframes

In [49]:
simplemap_df["postal_code"] = pd.to_numeric(simplemap_df["postal_code"])

In [50]:
redfin_df.dtypes

state           object
postal_code     object
2021           float64
dtype: object

In [51]:
redfin_df["postal_code"]=pd.to_numeric(redfin_df["postal_code"])

In [52]:
#merge redfin and simplemap dataframes
# Merge dataframes
frames1 = [redfin_df, simplemap_df]
redfin_df_cleaned = reduce(lambda left,right: pd.merge(left,right,on='postal_code'), frames1)
print(redfin_df_cleaned.shape)
redfin_df_cleaned.head()

(16857, 7)


,state,postal_code,2021,city,state_id,state_name,county_name
0,Ohio,45784,248810.0,Vincent,OH,Ohio,Washington
1,Texas,76114,261764.0,Fort Worth,TX,Texas,Tarrant
2,California,91405,663543.0,Van Nuys,CA,California,Los Angeles
3,Illinois,60157,419458.0,Medinah,IL,Illinois,DuPage
4,Iowa,50309,230289.0,Des Moines,IA,Iowa,Polk


#### Drop unnecessary columns

In [53]:
redfin_df_cleaned.columns

Index(['state', 'postal_code', '2021', 'city', 'state_id', 'state_name',
       'county_name'],
      dtype='object')

In [54]:
#drop unnnecessary columns
redfin_df_cleaned = redfin_df_cleaned.drop(columns = ["state_name", "state"])

In [55]:
redfin_df_cleaned.head()

,postal_code,2021,city,state_id,county_name
0,45784,248810.0,Vincent,OH,Washington
1,76114,261764.0,Fort Worth,TX,Tarrant
2,91405,663543.0,Van Nuys,CA,Los Angeles
3,60157,419458.0,Medinah,IL,DuPage
4,50309,230289.0,Des Moines,IA,Polk


#### Rename columns
state_id: "State", city:"City", county_name:"CountyName

In [56]:
# rename columns 
redfin_df_cleaned = redfin_df_cleaned.rename(columns={"state_id": "State", "city":"City", "county_name":"CountyName"})

#### Reorder columns

In [57]:
redfin_df_cleaned = redfin_df_cleaned[["postal_code","State", "City", "CountyName","2021"]]

In [58]:
print(redfin_df_cleaned.shape)
redfin_df_cleaned.head()

(16857, 5)


,postal_code,State,City,CountyName,2021
0,45784,OH,Vincent,Washington,248810.0
1,76114,TX,Fort Worth,Tarrant,261764.0
2,91405,CA,Van Nuys,Los Angeles,663543.0
3,60157,IL,Medinah,DuPage,419458.0
4,50309,IA,Des Moines,Polk,230289.0


# Merging zillow and redfin datasets

In [59]:
#load cleaned zillow dataset
zillow_df = pd.read_csv("zillow_housing_final_cleaned.csv")
print(zillow_df.shape)
zillow_df.head()

(30468, 6)


,Unnamed: 0,postal_code,State,City,CountyName,2021
0,0,10025,NY,New York,New York County,1114145
1,1,60657,IL,Chicago,Cook County,508407
2,2,10023,NY,New York,New York County,1450186
3,3,77494,TX,Katy,Harris County,404243
4,4,60614,IL,Chicago,Cook County,648360


In [60]:
zillow_col_drop = zillow_df.columns[0]
zillow_df = zillow_df.drop(zillow_col_drop, axis=1)
zillow_df.head()

,postal_code,State,City,CountyName,2021
0,10025,NY,New York,New York County,1114145
1,60657,IL,Chicago,Cook County,508407
2,10023,NY,New York,New York County,1450186
3,77494,TX,Katy,Harris County,404243
4,60614,IL,Chicago,Cook County,648360


In [61]:
zillow_df.shape

(30468, 5)

#### Append redfin and zillow datasets

In [62]:
merge_df = zillow_df.append(redfin_df_cleaned)
print(merge_df.shape)
merge_df.head()

(47325, 5)


,postal_code,State,City,CountyName,2021
0,10025,NY,New York,New York County,1114145.0
1,60657,IL,Chicago,Cook County,508407.0
2,10023,NY,New York,New York County,1450186.0
3,77494,TX,Katy,Harris County,404243.0
4,60614,IL,Chicago,Cook County,648360.0


In [63]:
#drop duplicates
merge_df = merge_df.drop_duplicates(["postal_code"])

In [64]:
merge_df.shape

(30645, 5)

# Load

In [66]:
#export to csv
merge_df.to_csv("zillow_redfin_merged.csv")